<center><h1>Exploring the Competitors</h1></center>

### Part 1. Get location data using Foursquare

[Foursquare Places API](https://location.foursquare.com/products/places-api/) is very usefule online application used by many developers & other application like Uber etc. In this project you can used it to retrieve informtion about the places present in the neighborhoods of Toronto. The API returns a JSON file and we need to turn that into a data-frame. Here I’ve chosen similar businesses(pet grooming) for each neighborhood within a radius of 2.5km.

You will need to create an account with Foursquare to access the API.  It is free to sign up, and you get $200 free credit.



In [1]:
import requests
import pandas as pd

c:\Users\Martin\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\Martin\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [8]:
toronto_DF = pd.read_csv('Toronto_postal_codes_and_coordinates.csv')

In [9]:
display(toronto_DF)

,Unnamed: 0,Borough,Postalcode,Neighborhood,Latitude,Longitude
0,0,Central Toronto,M4N,Lawrence Park,43.728020,-79.388790
1,1,Central Toronto,M4P,Davisville North,43.712751,-79.390197
2,2,Central Toronto,M4R,North Toronto West,43.715383,-79.405678
3,3,Central Toronto,M4S,Davisville,43.704324,-79.388790
4,4,Central Toronto,M4T,"Moore Park, Summerhill East",43.689574,-79.383160
...,...,...,...,...,...,...
98,98,York,M6C,Humewood-Cedarvale,43.693781,-79.428191
99,99,York,M6E,Caledonia-Fairbanks,43.689026,-79.453512
100,100,York,M6M,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",43.691116,-79.476013
101,101,York,M6N,"Runnymede, The Junction North",43.673185,-79.487262


In [14]:
# Set up your API key and other constants for Foursquare API
API_KEY = 'fsq3jAlNsDDxUztdezP1wMBdFBKS6ICOZlo2zlRdOu7D1nk='  # Replace with your actual Foursquare API key
LIMIT = 20  # Maximum number of venues to return
radius = 5000  # Search radius in meters

# a function to loop through each neighbourhood in the postalcode.csv file, and search for places
def getNearbyVenues(names, postalcode, latitudes, longitudes, radius):
    venues_list = []

    for name, postalcode, lat, lng in zip(names, postalcode, latitudes, longitudes):
        print(f"Processing neighborhood: {name}")

        # Skip if latitude or longitude is missing
        if pd.isnull(lat) or pd.isnull(lng):
            print(f"Skipping {name} due to missing latitude or longitude.")
            continue

        # Create the API request URL and parameters
        # Change the params here to customize your search, https://docs.foursquare.com/developer/reference/place-search
        # The following example looks for pet groomers in the neighbourhood
        # I am searching with categories ID 11134 - pet grooming services
        url = 'https://api.foursquare.com/v3/places/search'
        params = {
            'query': "pet store",
            'll': f'{lat},{lng}',
            'radius': radius,
            'limit': LIMIT,
        }

        # Set up headers with the API key
        headers = {
            "Accept": "application/json",
            "Authorization": API_KEY  
        }

        # Make the GET request
        response = requests.get(url, headers=headers, params=params)

        # Check if the request was successful
        if response.status_code != 200:
            print(f"Failed to get data for {name}. Status code: {response.status_code}")
            print(f"Error message: {response.text}")
            continue

        results = response.json()

        # Parse the JSON response
        for venue in results.get('results', []):
            # Extract venue information
            venue_name = venue.get('name')
            venue_id = venue.get('fsq_id')
            venue_location = venue.get('geocodes', {}).get('main', {})
            venue_lat = venue_location.get('latitude')
            venue_lng = venue_location.get('longitude')
            venue_categories = venue.get('categories', [])
            venue_category = venue_categories[0]['name'] if venue_categories else 'Unknown'

            # Append to the list
            venues_list.append([
                name,
                postalcode,
                lat,
                lng,
                venue_name,
                venue_id,
                venue_lat,
                venue_lng,
                venue_category
            ])

    # Create a DataFrame from the list
    nearby_venues = pd.DataFrame(venues_list, columns=[
        'Neighborhood',
        'Postalcode',
        'Neighborhood Latitude',
        'Neighborhood Longitude',
        'Venue',
        'fsq_id',
        'Venue Latitude',
        'Venue Longitude',
        'Venue Category'
    ])

    return nearby_venues

# Example usage with your DataFrame 'toronto_DF'
# Ensure 'toronto_DF' has the columns 'Neighborhood', 'Latitude', 'Longitude'
toronto_venues = getNearbyVenues(
    names=toronto_DF['Neighborhood'],
    postalcode=toronto_DF['Postalcode'],
    latitudes=toronto_DF['Latitude'],
    longitudes=toronto_DF['Longitude'],
    radius=radius
)

# Display the first few rows of the resulting DataFrame
toronto_venues.head()


Processing neighborhood: Lawrence Park
Processing neighborhood: Davisville North
Processing neighborhood: North Toronto West
Processing neighborhood: Davisville
Processing neighborhood: Moore Park, Summerhill East
Processing neighborhood: Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Processing neighborhood: Roselawn
Processing neighborhood: Forest Hill North & West
Processing neighborhood: The Annex, North Midtown, Yorkville
Processing neighborhood: Rosedale
Processing neighborhood: St. James Town, Cabbagetown
Processing neighborhood: Church and Wellesley
Processing neighborhood: Regent Park, Harbourfront
Processing neighborhood: Garden District, Ryerson
Processing neighborhood: St. James Town
Processing neighborhood: Berczy Park
Processing neighborhood: Central Bay Street
Processing neighborhood: Richmond, Adelaide, King
Processing neighborhood: Harbourfront East, Union Station, Toronto Islands
Processing neighborhood: Toronto Dominion Centre, Design Exchange
Proc

,Neighborhood,Postalcode,Neighborhood Latitude,Neighborhood Longitude,Venue,fsq_id,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,M4N,43.72802,-79.38879,PetSmart,580bec0d38fa83192d9e6bb5,43.716614,-79.446053,Pet Supplies Store
1,Lawrence Park,M4N,43.72802,-79.38879,PetSmart,5a0c5ec8fdb9a7258ac7f16d,43.769118,-79.412522,Pet Supplies Store
2,Lawrence Park,M4N,43.72802,-79.38879,Vetster Inc,60a25e64c1e6311b03a170ad,43.730266,-79.402867,Retail
3,Lawrence Park,M4N,43.72802,-79.38879,Bosley's,731468cd36ae7c837f0c58c8,43.732714,-79.403961,Pet Supplies Store
4,Lawrence Park,M4N,43.72802,-79.38879,Pet Valu,4b5b3743f964a520d8eb28e3,43.714116,-79.400148,Pet Supplies Store


In [15]:
# The total number of pet grooming services in Toronto
toronto_venues['Venue'].nunique()

199

In [5]:
# Oberserving the number of competitors in each neighbourhood
toronto_venues.groupby('Neighborhood')['Venue'].count()

Neighborhood
Agincourt                                          20
Alderwood, Long Branch                             20
Bathurst Manor, Wilson Heights, Downsview North    20
Bayview Village                                    20
Bedford Park, Lawrence Manor East                  20
                                                   ..
Willowdale, Newtonbrook                            20
Woburn                                             20
Woodbine Heights                                   20
York Mills West                                    20
York Mills, Silver Hills                           20
Name: Venue, Length: 103, dtype: int64

In [72]:
toronto_venues.to_excel("toronto_pet_venues.xlsx")

C:\Users\Martin\AppData\Local\Temp/ipykernel_32276/442604555.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.1' currently installed).
  toronto_venues.to_excel("toronto_pet_venues.xlsx")


### Part 2. Interactive leaflet map using coordinate data.


In [2]:
import folium # map rendering library

In [3]:
toronto_venues = pd.read_excel("Data Extract/toronto_pet_venues.xlsx")

In [25]:
# My example below shows pet groomers in Lawrence Park
target = ["M2J","M2N","M5A","M5V","M6H","M6K","M8V","M9V"]


search_area = toronto_venues[toronto_venues['Postalcode'].isin(target)]
latitude = toronto_DF[toronto_DF['Postalcode'].isin(target)]['Latitude'].mean()
longitude = toronto_DF[toronto_DF['Postalcode'].isin(target)]['Longitude'].mean()
display(search_area)
display(latitude)
# search_area = toronto_venues
# latitude = toronto_DF['Latitude'].mean()
# longitude = toronto_DF['Longitude'].mean()
# display(toronto_venues)

,Unnamed: 0,Neighborhood,Postalcode,Neighborhood Latitude,Neighborhood Longitude,Venue,fsq_id,Venue Latitude,Venue Longitude,Venue Category
240,240,"Regent Park, Harbourfront",M5A,43.654260,-79.360636,Pet Cuisine,4b646bedf964a520d5b12ae3,43.649770,-79.369924,Technology Business
241,241,"Regent Park, Harbourfront",M5A,43.654260,-79.360636,Helmutt's Pet Supply,4ae0a77cf964a520528121e3,43.645754,-79.410561,Pet Supplies Store
242,242,"Regent Park, Harbourfront",M5A,43.654260,-79.360636,Timmie Doggie Outfitters,4f2439f4e4b0020c2e8c6387,43.645754,-79.410561,Pet Grooming Service
243,243,"Regent Park, Harbourfront",M5A,43.654260,-79.360636,Pet Valu,4f39937fe4b0a4d442d0af8c,43.667128,-79.314509,Pet Supplies Store
244,244,"Regent Park, Harbourfront",M5A,43.654260,-79.360636,Pet Valu,4b7aebd9f964a5202f462fe3,43.657277,-79.402630,Pet Supplies Store
...,...,...,...,...,...,...,...,...,...,...
1869,1869,"Brockton, Parkdale Village, Exhibition Place",M6K,43.636847,-79.428191,Fetch,4c74461dff1fb60cde14f3a7,43.650201,-79.450401,Pet Supplies Store
1870,1870,"Brockton, Parkdale Village, Exhibition Place",M6K,43.636847,-79.428191,Umbrella Pet Svc,4b893c4cf964a520fb2432e3,43.642224,-79.399667,Pet Grooming Service
1871,1871,"Brockton, Parkdale Village, Exhibition Place",M6K,43.636847,-79.428191,Pet Valu,617ef4af37f74d634bb31c99,43.651694,-79.451482,Pet Supplies Store
1872,1872,"Brockton, Parkdale Village, Exhibition Place",M6K,43.636847,-79.428191,Bosley's,1bb15b357f48e1cbaa7b8ae9,43.651968,-79.451573,Pet Supplies Store


43.6853449125

In [26]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, venue, neighborhood in zip(search_area['Venue Latitude'], search_area['Venue Longitude'], search_area['Venue'], search_area['Neighborhood']):
    label = '{},{}'.format(venue, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

map_toronto

In [27]:
# Assuming `search_area` is a DataFrame with venue data
# Add a count column for each unique location
search_area['Count'] = search_area.groupby(['Venue Latitude', 'Venue Longitude'])['Venue'].transform('count')

# Create the map
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=14)

# Add markers to the map with size proportional to the count
for lat, lng, venue, neighborhood, count in zip(
    search_area['Venue Latitude'],
    search_area['Venue Longitude'],
    search_area['Venue'],
    search_area['Neighborhood'],
    search_area['Count']
):
    label = f"{venue}, {neighborhood} (Count: {count})"
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3 + count,  # Radius grows with count
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False
    ).add_to(map_toronto)

# Display the map
map_toronto

C:\Users\Martin\AppData\Local\Temp/ipykernel_5744/1798107268.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  search_area['Count'] = search_area.groupby(['Venue Latitude', 'Venue Longitude'])['Venue'].transform('count')


### Part 3. Customer Reviews
It seems like there is one one pet groomer withing 2km of Scarborough Village, I am interested in seeing customer's comments on Funny Bunny.  I can use [Foursquare's Place Tips API.](https://docs.foursquare.com/developer/reference/place-tips)


In [ ]:
# a function to loop through the list of pet groomers in the neighborhood and compile all the comments related to them
def getTips(venues, fsq_ids):
    venues_tips = []

    for venue, fsq_id in zip(venues, fsq_ids):
        print(f"Processing venue: {venue}")


        # Create the API request URL and parameters
        url = f"https://api.foursquare.com/v3/places/{fsq_id}/tips"

        headers = {
            "Accept": "application/json",
            "Authorization": 'fsq3jAlNsDDxUztdezP1wMBdFBKS6ICOZlo2zlRdOu7D1nk='   
        }
        
        # Make the GET request
        response = requests.get(url, headers=headers)

        # Check if the request was successful
        if response.status_code != 200:
            print(f"Failed to get data for {venue}. Status code: {response.status_code}")
            print(f"Error message: {response.text}")
            continue

        results = response.json()
        
        # Parse the JSON response
        for tip in results:
            # Extract venue information
            tip_id = tip.get('id')
            tip_created = tip.get('created_at')
            tip_text = tip.get('text')
   

            # Append to the list
            venues_tips.append([
                venues,
                tip_id,
                tip_created,
                tip_text
            ])
            
    
    # Create a DataFrame from the list
    nearby_tips = pd.DataFrame(venues_tips, columns=[
        'venue_name',
        'id',
        'created_at',
        'text'
    ])

    return nearby_tips

nearby_tips = getTips(
    venues=search_area['Venue'].unique(),
    fsq_ids=search_area['fsq_id']
)

# Display the first few rows of the resulting DataFrame
nearby_tips.head()

KeyError: "None of [Index(['Pet Cuisine', 'Helmutt's Pet Supply', 'Timmie Doggie Outfitters',\n       'Pet Valu', 'Knick Knack Paddywhack Pet Store & Grooming', 'Bosley's',\n       'Wongs Aquarium', 'The Dog House', 'Furry Friends Training',\n       'Feed Me More Pet Foods', 'Gentle Paw', 'Petey's Grooming & Treats',\n       'K-9 Couture', 'Paulmac's', 'Bini Pets', 'Global Pet Foods', 'Petview',\n       'Pet Uno', 'PetSmart', 'Pet Social', 'Umbrella Pet Svc',\n       'Pearly Whites For Pets', 'The Bone & Biscuit Co',\n       'Market Street Pharmacy', 'Urban Dog Products Inc', 'Pet Stuff',\n       'BooBoo Pet Grooming Svc', 'Kennel Cafe', 'Purrfect Pet Supplies',\n       'Pet Pointe', 'Bone & Biscuit', '2Bone Appetit Long Branch',\n       'Bone Appetit Long Branch', 'Poochykat', 'Healthy Paws',\n       'Carl's Aquarium Servicing', 'Vinces Pet Centre Ltd',\n       'Star Aquarium Products Ltd', 'Calm and Balanced Canine',\n       'Kikimo Pet基基猫', 'Yu Pet Hotel', 'Global Pet Foods Store',\

In [35]:
len(search_area['Venue'].unique())

60

In [30]:
nearby_tips.to_excel("tips.xlsx")

C:\Users\Martin\AppData\Local\Temp/ipykernel_5744/2135313292.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.1' currently installed).
  nearby_tips.to_excel("tips.xlsx")


### Now it is your turn to discover interesting search results using Foursquare
You are expected to read through the API documentation and customize the searches.  